<a href="https://colab.research.google.com/github/dellavecchiaemiliano/Econometrics_for_finance/blob/main/CNN_F_MNIST_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IMPORTING LIBRARIES**

In [ ]:
%matplotlib inline

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.nn.functional as F

# **Downloading data**

First, I download the dataset fashion-MNIST which contains 60000 training images and 10000 test images of fashion and clothing items taken from 10 classes. Each image is a standardized 28x28 size in grayscale (784 total pixels). Then I transform the data into a tensor which is a multidimensional matrix.

In [ ]:
train_set = datasets.FashionMNIST("data", train=True, download=True, transform=ToTensor(),)
test_set = datasets.FashionMNIST("data", train=False, download=True, transform=ToTensor(),)

Here below, I divide my dataset into intervals of 64 samples in order to make faster the iteration and to reduce the variance of the model.

In [ ]:
batch_size = 64
train_dataloader = DataLoader(train_set, batch_size=batch_size)
test_dataloader = DataLoader(test_set, batch_size=batch_size)

# **Building CNN**
Finally, I design the structure of the CNN. The number of input is given by the quality of the images, in this case a matrix of 28x28, then I set two hidden layers with 512 nodes that return 10 outputs, each output represent a class. In order to make faster the algorithm I use the ReLu as activation function.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
            )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


# **Training**
At first, I specify the type of cost function, the type of gradient and the learning rate to take into account. Then I create the function "train" which minimize the cost function by using the backpropagation tool.  

In [ ]:
cost_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

In [ ]:
def train(dataloader, model, cost_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        cost = cost_fn(pred, y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            cost, current = cost.item(), batch * len(X)
            print(f"loss: {cost:>7f}  [{current:>5d}/{size:>5d}]")

# **Testing**
Now, I set the model in .eval() mode. It no longer compute the gradient and it returns the accuracy and the average loss of the model on the test set.

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_cost, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_cost += cost_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_cost /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_cost:>8f} \n")

In [ ]:
epochs = 50
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, cost_fn, optimizer)
    test(test_dataloader, model, cost_fn)

Epoch 1
-------------------------------
loss: 2.309425  [    0/60000]
loss: 2.168519  [ 6400/60000]
loss: 1.802842  [12800/60000]
loss: 1.496882  [19200/60000]
loss: 1.133817  [25600/60000]
loss: 1.033809  [32000/60000]
loss: 1.003939  [38400/60000]
loss: 0.858092  [44800/60000]
loss: 0.865884  [51200/60000]
loss: 0.803831  [57600/60000]
Test Error: 
 Accuracy: 71.7%, Avg loss: 0.786845 

Epoch 2
-------------------------------
loss: 0.789316  [    0/60000]
loss: 0.831269  [ 6400/60000]
loss: 0.581631  [12800/60000]
loss: 0.774856  [19200/60000]
loss: 0.671326  [25600/60000]
loss: 0.635321  [32000/60000]
loss: 0.709007  [38400/60000]
loss: 0.676740  [44800/60000]
loss: 0.703037  [51200/60000]
loss: 0.634546  [57600/60000]
Test Error: 
 Accuracy: 77.9%, Avg loss: 0.630151 

Epoch 3
-------------------------------
loss: 0.560955  [    0/60000]
loss: 0.642537  [ 6400/60000]
loss: 0.442320  [12800/60000]
loss: 0.665561  [19200/60000]
loss: 0.586767  [25600/60000]
loss: 0.557302  [32000/600

# **Saving the model**

In [ ]:
torch.save(model.state_dict(), "model.pth")

In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

# **Proof on an instance**


In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_set[0][0], test_set[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
